In [50]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'
import timeit
from datetime import datetime

In [2]:
# tf.Graph contains objects of tf.Operation and objects tf.Tensor flowing between them
# Models in tf are saved as Graph. 

In [9]:
# How to create/trace a graph in tf -> use tf.function directly or as decorator

# some python function
def function_to_get_faster(x, y, b):
    x = tf.matmul(x, y)
    x = x + b
    return x

# create tf.Funtion object -> convert function to graph now!
a_function_traced_to_graph = tf.function(function_to_get_faster)

# make some tf.Tensors
x1 = tf.constant([[1.0, 2.0]])
y1 = tf.constant([[2.0], [3.0]])
b1 = tf.constant(4.0)

a_function_traced_to_graph(x1, y1, b1).numpy().tolist()[0][0]

# some decorated python function
@tf.function
def decorated_function_to_get_faster(x, y, b):
    x = tf.matmul(x, y)
    x = x + b
    if b1.numpy()<0:
        return x
    else:
        return x+tf.constant(0.0)

# or loops of function is also fine to create graph. Any loops are also fine. Graph will 
# def func1(..): return func2(..)  ... def func2(..): return something

decorated_function_to_get_faster(x1, y1, b1).numpy()

12.0

array([[12.]], dtype=float32)

In [27]:
# speeding up with tf.Graph
class SequentialModel(tf.keras.Model):
    def __init__(self, **kwargs):
        super(SequentialModel, self).__init__(**kwargs)
        # following will convert (batch_size, a, b,c ) -> (batch_size,a*b*c)
        self.flatten = tf.keras.layers.Flatten()
        # add lot of layers
        num_layers = 100
        self.inner_layers = [tf.keras.layers.Dense(64, activation='relu') for n in range(num_layers)]
        self.dropout = tf.keras.layers.Dropout(0.2)
        self.outer_layer = tf.keras.layers.Dense(10)
    
    # @tf.function will trace a graph
    def call(self, x):
        x = self.flatten(x)
        for layer in self.inner_layers:
            x = layer(x)
        x = self.dropout(x)
        x = self.outer_layer(x)
        return x
        

In [28]:
input_data = tf.random.uniform([25, 28, 28])

In [35]:
# without graph
eager_model = SequentialModel()
eager_model(input_data)
print("Eager time:", timeit.timeit(lambda: eager_model(input_data), number=100))

<tf.Tensor: shape=(25, 10), dtype=float32, numpy=
array([[ 5.72243217e-18, -1.18814767e-16, -6.70611734e-17,
        -3.88738227e-17, -1.68027696e-17,  9.48875342e-18,
        -5.20921358e-17,  6.86947625e-17, -9.63800211e-17,
        -6.45736229e-17],
       [ 6.17613536e-18, -1.48091879e-16, -8.07584439e-17,
        -4.79163858e-17, -1.97955272e-17,  1.55454021e-17,
        -6.56110199e-17,  8.59882239e-17, -1.22156617e-16,
        -8.13277692e-17],
       [ 2.85178292e-18, -9.13426368e-17, -5.07892668e-17,
        -2.88825993e-17, -1.23308451e-17,  8.83083546e-18,
        -4.10288532e-17,  5.25580503e-17, -7.45246785e-17,
        -4.95432251e-17],
       [ 6.90795363e-18, -1.34782357e-16, -7.33232615e-17,
        -4.49410635e-17, -1.76797002e-17,  1.37441295e-17,
        -6.08746036e-17,  7.88622548e-17, -1.11690241e-16,
        -7.43601820e-17],
       [ 6.38639185e-18, -1.37612546e-16, -7.63345298e-17,
        -4.77203804e-17, -1.71819972e-17,  1.25086302e-17,
        -6.51700929e

Eager time: 3.9543137999989995


In [41]:
# with tracing graph
eager_model_with_graph = SequentialModel()
eager_model_with_graph.call = tf.function(eager_model_with_graph.call)

print("Eager time:", timeit.timeit(lambda: eager_model_with_graph(input_data), number=100))

Eager time: 3.060538200001247


In [48]:
# when you do tf.function() (== a_function) -> you make a polymorphic function which has several graph functions

def afunction(a):
    return tf.constant(a)

print(tf.function(lambda x: tf.constant(x)))


In [52]:
# identity layer with eager side-effects
class EagerLayer(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super(EagerLayer, self).__init__()
        
    def call(self,inputs):
        print("Eager called ",str(datetime.now()))
        return inputs
    

class SequentialModel(tf.keras.Model):
  def __init__(self):
    super(SequentialModel, self).__init__()
    self.flatten = tf.keras.layers.Flatten()
    self.dense_1 = tf.keras.layers.Dense(128, activation="relu")
    self.dropout = tf.keras.layers.Dropout(0.2)
    self.dense_2 = tf.keras.layers.Dense(10)
    self.eager = EagerLayer()

  def call(self, x):
    x = self.flatten(x)
    x = self.dense_1(x)
    x = self.dropout(x)
    x = self.dense_2(x)
    return self.eager(x)
    
input_data = tf.random.uniform([60, 28, 28])
labels = tf.random.uniform([60])

loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    
model = SequentialModel()
# set loss function, optimization, etc. Nothing about graph tracing here
model.compile(run_eagerly=False, loss=loss_fn)
model.fit(input_data, labels, epochs=1)

    

Eager called  2021-01-01 23:01:28.591233
Eager called  2021-01-01 23:01:29.082197
2/2 [==============================] - 0s 6ms/step - loss: 1.2675


In [54]:
# Now, globally set everything to run eagerly
tf.config.run_functions_eagerly(True)
print("Run all functions eagerly.")

# Create a polymorphic function
polymorphic_function = tf.function(model)

print("\nCalling twice eagerly")
# When you run the function again, you will see the side effect
# twice, as the function is running eagerly.
result = polymorphic_function(input_data)
result = polymorphic_function(input_data)

Run all functions eagerly.

Calling twice eagerly
Eager called  2021-01-01 23:02:34.986816
Eager called  2021-01-01 23:02:34.991300


In [55]:
# Don't forget to set it back when you are done
tf.config.experimental_run_functions_eagerly(False)


Instructions for updating:
Use `tf.config.run_functions_eagerly` instead of the experimental version.
